## libraries and function

In [ ]:
!pip install fancyimpute

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 1.1 MB/s eta 0:00:00
  Created wheel for fancyimpute: filename=fancyimpute-0.7.0-py3-none-any.whl size=29880 sha256=1ee2d4cfbd1cd859178adfce2e970cc456a2559b2af11ca7994ca91a32776cc3
  Stored in directory: /root/.cache/pip/wheels/7b/0c/d3/ee82d1fbdcc0858d96434af108608d01703505d453720c84ed
  Created wheel for knnimpute: filename=knnimpute-0.1.0-py3-none-any.whl size=11329 sha256=eff7d5542a58c41f49d87371275c343b84a5adf6c6c929554b7df0b2405bd817
  Stored in directory: /root/.cache/pip/wheels/46/06/a5/45a724630562413c374e29c08732411d496092408b3a7bf754
Successfully built fancyimpute knnimpute


In [ ]:
!pip install DistributedMissForest

  Preparing metadata (setup.py) ... done
  Created wheel for DistributedMissForest: filename=DistributedMissForest-1.4-py3-none-any.whl size=17469 sha256=a9775d5e77a3e3f80e255dce24a48d1cb201d217220268b87af1f53458fe2ce8
  Stored in directory: /root/.cache/pip/wheels/62/93/ad/606370d3635fc2b977fafeeccd59929f886b09f19d4386e0ac
Successfully built DistributedMissForest


In [ ]:
!pip install missingpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 489.9 kB/s eta 0:00:00


In [ ]:
!pip install MissForest

In [ ]:
import time
import math
import numpy as np
import pandas as pd
import sklearn.neighbors._base
import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from sklearn import datasets
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as skLDA
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from scipy import stats
from fancyimpute import SoftImpute
from missforest.missforest import MissForest
from sklearn.model_selection import train_test_split
from numpy.linalg import norm, inv, det

In [ ]:
def diag_term(i, X, y, G):
    arr0 = X[:, i]
    arr = arr0[~np.isnan(arr0)]
    y_arr = y[~np.isnan(arr0)]

    _, counts = np.unique(y_arr, return_counts = True)
    ind = np.insert(np.cumsum(counts), 0, 0)

    return sum([(ind[g] - ind[g - 1]) * np.var(arr[y_arr == g - 1]) for
                g in range(1, G + 1)]) / len(y_arr)


def dper(X, y, G):
    '''
    X: input, should be a numpy array
    y: label
    G: number of classes
    output:
    - mus: each row is a class mean
    - S: common covariance matrix of class 1,2,..., G
    '''
    epsilon = 1e-5  # define epsilon to put r down to 0 if r < epsilon
    n, p = X.shape[0], X.shape[1]

    mus = np.array(
        [np.nanmean(X[y == g, :], axis = 0) for g in range(G)]).T  # so that each column is the mean of a class

    S = np.diag([diag_term(i, X, y, G) for i in range(p)])

    for i in range(p):
        for j in range(i):
            mat = X[:, [i, j]]

            # drop rows with NA
            idx = ~np.isnan(mat).any(axis = 1)
            mat, y_arr = mat[idx], y[idx]

            _, counts = np.unique(y_arr, return_counts = True)
            ind = np.insert(np.cumsum(counts), 0, 0)

            m_g = counts

            A = len(y_arr)

            scaled_mat = [mat[y_arr == g, :] - mus[[i, j], g] for g in range(G)]

            q = lambda g: np.dot(scaled_mat[g][:, 0], scaled_mat[g][:, 0])
            s11 = sum(map(q, range(G)))
            q = lambda g: np.dot(scaled_mat[g][:, 1], scaled_mat[g][:, 1])
            s22 = sum(map(q, range(G)))
            d = lambda g: np.dot(scaled_mat[g][:, 0], scaled_mat[g][:, 1])
            s12 = sum(map(d, range(G)))

            start_solve = time.time()
            B = S[i, i] * S[j, j] * A - s22 * S[i, i] - s11 * S[j, j]
            coefficient = [-A, s12, B, s12 * S[i, i] * S[j, j]]
            r = np.roots(coefficient)

            r = r[abs(np.imag(r)) < epsilon]
            r = np.real(r)
            r[abs(r) < epsilon] = 0

            if len(r) > 1:
                condi_var = S[j, j] - r ** 2 / S[i, i]
                eta = -A * np.log(condi_var) - (S[j, j] - 2 * r / S[i, i] * s12 +
                                                r ** 2 / S[i, i] ** 2 * s11) / condi_var
                # if condi_var <0 then eta = NA. in practice, it's impossible for cov to be negative
                #  therefore, we drop NA elements of eta
                r = r[eta == max(eta[~np.isnan(eta)])]

            if len(r) > 1:
                w = [m_g[g - 1] * np.cov(mat[ind[g - 1]:ind[g], ], rowvar = False) for
                     g in range(1, G + 1)]
                w = np.sum(w, axis = 0)
                r = r[np.abs(r - w[0, 1]).argmin()]  # choose r that is w[0,1]

            S[i, j] = S[j, i] = r
    return [mus, S]

In [ ]:
# @title
import numpy as np

def generate_nan(X, non_missing , missing_rate):
    X_copy=np.copy(X)

    X_non_missing_col = X_copy[:, non_missing]
    X1_missing = X_copy[:, [i for i in range(X.shape[1]) if i not in non_missing]]

    X_non_missing_row = X1_missing[non_missing]
    X_missing = X1_missing[len(non_missing):(X.shape[0]+1)]

    XmShape = X_missing.shape
    na_id = np.random.randint(0, X_missing.size, round(missing_rate * X_missing.size))
    X_nan = X_missing.flatten()
    X_nan[na_id] = np.nan
    X_nan = X_nan.reshape(XmShape)

    X1_nan = np.vstack((X_non_missing_row, X_nan))
    X_nan = np.hstack((X_non_missing_col, X1_nan))
    return X_nan

def PHF(Xm, y, G):
    p = Xm.shape[1]
    non_missing_id = np.where(np.sum(np.isnan(Xm), axis = 0) == 0)
    q = max(non_missing_id)[-1] + 1
    # initialize the parameter with 0s
    S = np.zeros((p, p))
    # estimating the parameter for the fully observed features
    mu = np.array([np.nanmean(Xm[y == g, :], axis = 0) for g in range(G)]).T  # the mean is stored column
    S[:q, :q] = sum([sum(y == g) * np.cov(Xm[y == g, :q], rowvar = False, ddof = 0) for g in range(G)]) / len(y)
    # estimating the parameter in step 2 of the algorithm:
    for j in range(q, p):
        id = ~np.isnan(Xm[:, j])
        Xfj = Xm[id, :][:, np.hstack((range(q), j))]
        yfj = y[id]
        covm = sum([sum(yfj == g) * np.cov(Xfj[yfj == g, :], rowvar = False, ddof = 0) for g in range(G)]) / len(yfj)
        Q = covm[-1, range(q)]  # row vector
        P = np.matmul(Q, np.linalg.inv(covm[:q, :q]))
        S[j, range(q)] = np.matmul(P, S[:q, :q])

        Xj, yj = Xm[id, j], y[id]
        cal_S = lambda g: np.dot(Xj[yj == g] - mu[j, g], Xj[yj == g] - mu[j, g])
        S[j, j] = sum(map(cal_S, range(G)))/len(yj)

        S[j, range(q, j)] = np.array([dper_sigij(Xm, y, G, mu, i, j, S[i, i], S[j, j]) for i in range(q, j)]).flatten()
        S[:, j] = S[j, :].T
    return mu, S


def dper_sigij(X, y, G, mus, i, j, sigii, sigjj):
    """
    X: input, should be a numpy array
    y: label
    G: number of classes
    i, j: index of the column
    sigii: sigma_ii
    sigjj: sigma_jj
    """
    epsilon = 1e-5  # define epsilon to put r down to 0 if r < epsilon

    mat = X[:, [i, j]]

    # drop rows with NA
    idx = ~np.isnan(mat).any(axis = 1)
    mat, y_arr = mat[idx], y[idx]

    A = len(y_arr)
    scaled_mat = [mat[y_arr == g, :] - mus[[i, j], g] for g in range(G)]
    q = lambda g: np.dot(scaled_mat[g][:, 0], scaled_mat[g][:, 0])
    s11 = sum(map(q, range(G)))
    q = lambda g: np.dot(scaled_mat[g][:, 1], scaled_mat[g][:, 1])
    s22 = sum(map(q, range(G)))
    d = lambda g: np.dot(scaled_mat[g][:, 0], scaled_mat[g][:, 1])
    s12 = sum(map(d, range(G)))
    B = sigii * sigjj * A - s22 * sigii - s11 * sigjj
    coefficient = [-A, s12, B, s12 * sigii * sigjj]
    r = np.roots(coefficient)

    r = r[abs(np.imag(r)) < epsilon]
    r = np.real(r)
    r[abs(r) < epsilon] = 0

    if len(r) > 1:
        condi_var = sigjj - r ** 2 / sigii
        # print('condi_var',condi_var)
        eta = -A * np.log(condi_var) - (sigjj - 2 * r / sigii * s12 +
                                        r ** 2 / sigii ** 2 * s11) / condi_var
        # if condi_var <0 then eta = NA. in practice, it's impossible for cov to be negative
        #  therefore, we drop NA elements of eta
        r = r[eta == max(eta[~np.isnan(eta)])]

    if len(r) > 1:
        r = r[np.abs(r - s12 / A).argmin()]  # select r that is closet to w[0,1]

    return r


def PHF1Cl(Xm):
  p = Xm.shape[1]
  non_missing_id = np.where(np.sum(np.isnan(Xm), axis=0)==0)
  q = max(non_missing_id)[-1] + 1
  # initialize the parameter with 0s
  mu = np.zeros((p, 1))
  S = np.zeros((p,p))
  # estimating the parameter for the fully observed features
  mu[:q,:] = np.mean(Xm[:,:q], axis = 0).reshape((-1,1)) # the mean is stored column
  S[:q,:q] =   np.cov(Xm[:, :q], rowvar = False, ddof = 0)
  # estimating the parameter in step 2 of the algorithm:
  for j in range(q,p):
    id = ~np.isnan(Xm[:,j])
    Xfj = Xm[id,:][:,np.hstack((range(q),j))]
    covm = np.cov(Xfj, rowvar = False, ddof = 0)
    Q = covm[-1,range(q)] #row vector
    P = np.matmul(Q, np.linalg.inv(covm[:q,:q]))
    S[j,range(q)]  = np.matmul(P, S[:q,:q])

    idj = ~np.isnan(Xm[:,j])
    Xj = Xm[idj,j]
    mu[j,:] =  np.nanmean(Xm[:,j], axis = 0)
    S[j,j] = sum((Xj - mu[j])**2) /len(Xj)

    S[j, range(q,j)] = np.array([dper_sigij1Cl(Xm, mu, i,j, S[i,i], S[j,j]) for i in range(q,j)]).flatten()
    S[:,j] = S[j, :].T
  return mu, S

def dper_sigij1Cl(X, mus, i,j, sigii, sigjj):
        '''
        X: input, should be a numpy array
        y: label
        G: number of classes
        i, j: index of the column
        sigii: sigma_ii
        sigjj: sigma_jj
        '''
        epsilon = 1e-5 # define epsilon to put r down to 0 if r < epsilon
        n,p = X.shape[0], X.shape[1]

        mat = X[:,[i,j]]

        # drop rows with NA
        idx = ~np.isnan(mat).any(axis=1)
        mat = mat[idx]

        A = len(mat)
        scaled_mat = mat-mus[[i,j]].T
        scaled_mat = np.vstack(scaled_mat)
        s11 = sum(scaled_mat[:,0]**2)
        s22 = sum(scaled_mat[:,1]**2)
        s12 = sum(scaled_mat[:,0]*scaled_mat[:,1])

        B = sigii*sigjj*A - s22 * sigii - s11 * sigjj
        coefficient = [-A, s12, B, s12*sigii*sigjj]
        r = np.roots(coefficient)

        r = r[abs(np.imag(r)) < epsilon]
        r = np.real(r)
        r[abs(r) < epsilon] = 0

        if len(r)>1:
          condi_var = sigjj - r**2/sigii
          # print('condi_var',condi_var)
          eta = -A*np.log(condi_var)-(sigjj-2*r/sigii*s12 +
                                      r**2/sigii**2*s11)/condi_var
          # if condi_var <0 then eta = NA. in practice, it's impossible for cov to be negative
          #  therefore, we drop NA elements of eta
          r = r[eta == max(eta[~np.isnan(eta)])]

        if len(r) > 1:
            r = r[np.abs(r-s12/A).argmin()] # select r that is closet to w[0,1]

        return r

In [ ]:
def l2err_inv(Strue, S):
    p = S.shape[0]
    return (norm(inv(Strue)-inv(S))/(p*p))

def l2err(Strue, S):
    p = S.shape[0]
    return (norm((Strue)-(S)) / (p*p))

def normalize_data(X):
  scaler = StandardScaler()
  scaler.fit(X)
  return scaler.transform(X)

In [ ]:
def experiment(X,y, n, missing_rate):
    G = len(np.unique(y))
    S0=np.cov(X.T)
    #print(S0.shape)
    err = []
    i_err=[]
    for i in range(n):
        mu_0=np.array([np.mean(X[y==g,:])  for g in np.unique(y) ]).T
        Xm0 = generate_nan(X, [0,1], missing_rate)
        Xm=Xm0
        muPhf, SPhf = PHF(Xm, y, G)

        muDper, SDper = dper(Xm, y, G)

        XMice= IterativeImputer(max_iter=100).fit(Xm).transform(Xm)
        SMice =  sum([sum(y==g)*np.cov(XMice[y==g, :], rowvar = False) for g in range(G)])/len(y)

        Xm_copy= pd.DataFrame.from_records(Xm)
        Missf= MissForest()
        XMissf_df = Missf.fit_transform(Xm_copy)
        XMissf= XMissf_df.to_numpy()
        SMissf =  sum([sum(y==g)*np.cov(XMissf[y==g, :], rowvar = False) for g in range(G)])/len(y)

        XSoft =  SoftImpute(max_iters = 100, verbose = False).fit_transform(Xm)
        SSoft =  sum([sum(y==g)*np.cov(XSoft[y==g, :], rowvar = False) for g in range(G)])/len(y)

#       Xxgb = XGBImputer().fit_transform(Xm)
#       Sxgb = sum([sum(y==g)*np.cov(Xxgb[y==g, :], rowvar = False) for g in range(G)])/len(y)

        i_err.append(np.array([l2err_inv(S0, SPhf), l2err_inv(S0, SDper),
                             l2err_inv(S0, SMice), l2err_inv(S0, SMissf),
                             l2err_inv(S0, SSoft)]))

        err.append(np.array([l2err(S0, SPhf), l2err(S0, SDper),
                             l2err(S0, SMice), l2err(S0, SMissf),
                             l2err(S0, SSoft)]))
    return i_err, err

#Ionosphere



In [ ]:
data = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/ionosphere/ionosphere.data',
                  sep = ",", header = None)
data.head()
data = pd.DataFrame.to_numpy(data)
X, y = data[:,:34].astype(np.float64), data[:,34]
le2 = LabelEncoder()
y = le2.fit_transform(y)
G = len(np.unique(y))
X = np.delete(X,[0,1], axis = 1)
for g in range(G):
  print(sum(y==g))
X.shape
X = normalize_data(X)
S = np.cov(X, rowvar = False)
print(norm(S))
print(norm(inv(S)))
X.shape

In [ ]:
num = 10
e_inv20,e20 = experiment(X,y,num, missing_rate=.2)
e_inv40,e40 = experiment(X,y,num, missing_rate=.4)
e_inv60,e60 = experiment(X,y,num, missing_rate=.6)
e_inv80,e80 = experiment(X,y,num, missing_rate=.8)

In [ ]:
print('err of PHF, DPER, Mice, MissForest, Soft-Impute')
np.vstack((np.mean(e20, axis = 0), np.mean(e40, axis = 0),
          np.mean(e60, axis = 0),np.mean(e80, axis = 0))).round(3)

err of PHF, DPER, Mice, MissForest, Soft-Impute


array([[0.002, 0.002, 0.002, 0.004, 0.002],
       [0.003, 0.003, 0.003, 0.006, 0.003],
       [0.003, 0.003, 0.003, 0.007, 0.004],
       [0.004, 0.004, 0.004, 0.008, 0.005]])

In [ ]:
print('std of PHF, DPER, Mice, MissForest, Soft-Impute')
np.vstack((np.std(e20, axis = 0), np.std(e40, axis = 0),
          np.std(e60, axis = 0),np.std(e80, axis = 0))).round(3)

std of PHF, DPER, Mice, MissForest, Soft-Impute


array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [ ]:
print('inv err of PHF, DPER, Mice, MissForest, Soft-Impute')
np.vstack((np.mean(e_inv20, axis = 0), np.mean(e_inv40, axis = 0),
          np.mean(e_inv60, axis = 0),np.mean(e_inv80, axis = 0)))

inv err of PHF, DPER, Mice, MissForest, Soft-Impute


array([[5.47383166e+00, 3.64511117e-01, 1.97447779e+07, 1.26140747e-02,
        1.79638366e-02],
       [1.20651005e-01, 1.75555407e-01, 1.72841277e+12, 1.49283892e-02,
        3.88629677e-02],
       [4.82442403e-01, 1.34399095e+00, 1.55684404e+12, 1.54654331e-02,
        9.16412456e-02],
       [1.80988438e-01, 3.58996952e-01, 4.55901423e+12, 1.54424993e-02,
        2.22570004e-01]])

In [ ]:
print('inv std of PHF, DPER, Mice, MissForest, Soft-Impute')
np.vstack((np.std(e_inv20, axis = 0), np.std(e_inv40, axis = 0),
          np.std(e_inv60, axis = 0),np.std(e_inv80,axis = 0))).round(3)

inv std of PHF, DPER, Mice, MissForest, Soft-Impute


array([[1.58410000e+01, 5.44000000e-01, 5.78638454e+07, 1.00000000e-03,
        2.00000000e-03],
       [7.90000000e-02, 1.42000000e-01, 4.46249971e+12, 0.00000000e+00,
        5.00000000e-03],
       [7.15000000e-01, 3.20700000e+00, 2.55247781e+12, 0.00000000e+00,
        1.70000000e-02],
       [1.77000000e-01, 7.76000000e-01, 8.43368178e+12, 0.00000000e+00,
        6.20000000e-02]])